In [36]:
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd

In [37]:

# Specify file path of raw data
path = 'C:/Users/33769/OneDrive/Bureau/Learning/2. Python/2. Dash/PPD/'
file_path = path + 'pp-complete.csv'

# Define column names
column_names = ['transaction_id','price', 'transfer_date', 'postcode', 'property_type', 'new_build_flag', 'duration', 'primary_address', 'secondary_address', 'street', 'locality', 'city', 'district', 'county', 'ppd_category', 'record_status']

# Specify data types for each column
dtypes = dict.fromkeys(column_names, 'string')
dtypes.update({'price': 'int64'})

# Define columns we don't want
usecols = [col for col in column_names if col != 'transaction_id']

# Read CSV using Dask with specified data types and parse_dates
data = dd.read_csv(file_path,header=None, names=column_names,dtype=dtypes,usecols=usecols)

# Compute to get a Pandas DataFrame
df = data.compute()

In [38]:
df.head()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,county,ppd_category,record_status
0,42000,1995-12-21 00:00,NE4 9DN,S,N,F,8,<NA>,MATFEN PLACE,FENHAM,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,A,A
1,95000,1995-03-03 00:00,RM16 4UR,S,N,F,30,<NA>,HEATH ROAD,GRAYS,GRAYS,THURROCK,THURROCK,A,A
2,74950,1995-10-03 00:00,CW10 9ES,D,Y,F,15,<NA>,SHROPSHIRE CLOSE,MIDDLEWICH,MIDDLEWICH,CONGLETON,CHESHIRE,A,A
3,43500,1995-11-14 00:00,TS23 3LA,S,N,F,19,<NA>,SLEDMERE CLOSE,BILLINGHAM,BILLINGHAM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
4,63000,1995-09-08 00:00,CA25 5QH,S,N,F,8,<NA>,CROSSINGS CLOSE,CLEATOR MOOR,CLEATOR MOOR,COPELAND,CUMBRIA,A,A


In [39]:
df['transfer_date'] = dd.to_datetime(df['transfer_date'], errors='coerce')

In [40]:
df.dtypes

price                          int64
transfer_date         datetime64[ns]
postcode             string[pyarrow]
property_type        string[pyarrow]
new_build_flag       string[pyarrow]
duration             string[pyarrow]
primary_address      string[pyarrow]
secondary_address    string[pyarrow]
street               string[pyarrow]
locality             string[pyarrow]
city                 string[pyarrow]
district             string[pyarrow]
county               string[pyarrow]
ppd_category         string[pyarrow]
record_status        string[pyarrow]
dtype: object

In [42]:
# Filtrer uniquement les années 2024 et 2025
df = df[df['transfer_date'].dt.year.isin(range(2020, 2026))]

# Réinitialiser l'index proprement
df = df.reset_index(drop=True)

In [ ]:



# Drop rows with no postcode
df = df.dropna(subset=['postcode'])

#Exclude property type Other (O)
df = df.loc[df['property_type']!='O']

# Sort by Date
#df.sort_values(by=['transfer_date'], inplace=True, ignore_index=True)

In [ ]:
df.shape

(1207785, 15)

Add postcode latitude and longitude


In [ ]:
# Read the postcode CSV file
postcodes_df = pd.read_csv('ukpostcodes.csv')

# Convert df to dict for faster lookup
postcodes = dict()
for (postcode, latitude, longitude) in postcodes_df[['postcode', 'latitude', 'longitude']].values:postcodes[postcode] = [latitude, longitude]

# Apply the mapping to create the new column
df['postcode_lat_long'] = df['postcode'].map(postcodes)

In [ ]:
# Add postcode sectors
df['postcode_sector'] =  df['postcode'].apply(lambda x: x[:x.find(' ')+2])


Add region column as defined by GeoJSON files


In [ ]:
# Load regions by postcodes
regions_df = pd.read_csv('postcode_areas.csv', usecols=['postcode_prefix', 'region'])

# Create a dictionary from the DataFrame
postcode_region_dict = dict(zip(regions_df['postcode_prefix'], regions_df['region']))

# Add postcode prefix column to main df
df['postcode_prefix'] = df['postcode'].str.extract(r'^([A-Z]+)')

# Map the postcode prefix to the region using the dictionary
df['region'] = df['postcode_prefix'].map(postcode_region_dict)

# Drop the intermediate 'postcode_prefix' column
df.drop(columns=['postcode_prefix'], inplace=True)

#Exclude sales in Scotland as limited data available for this region
df = df.loc[df['region']!='Scotland']

In [ ]:
#Add year column
# Extract the year from 'transfer_date'
df['year'] = df['transfer_date'].dt.year

# Extract the month from 'transfer_date'
df['month'] = df['transfer_date'].dt.month

Create GeoJSON files


In [ ]:
# Create GeoJSON file for all sectors using Shapefil
myshpfile = gpd.read_file(path+'shapefiles/Sectors.shp')
myshpfile.to_file(path+'geo_json/all_sectors.geojson', driver='GeoJSON')

c:\Users\33769\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [ ]:
# Load the postcode prefixes CSV
postcode_prefixes_df = pd.read_csv('postcode_areas.csv')

# Load the GeoJSON file containing postcode sector data
geojson_path = 'geo_json/all_sectors.geojson'
postcode_sectors_gdf = gpd.read_file(geojson_path)

# Create an empty dictionary to store GeoDataFrames for each region
region_geojson_dict = {}

# Extract postcode prefix from 'name' property
postcode_sectors_gdf['postcode_prefix'] = pd.Series(postcode_sectors_gdf['name']).str.extract(r'^([A-Z]+)')

# Merge with postcode prefixes DataFrame to get regions
merged_df = pd.merge(postcode_sectors_gdf, postcode_prefixes_df, how='left', on='postcode_prefix')

# Create a dictionary to store GeoDataFrames for each region
region_geojson_dict = {}

# Iterate over unique regions
for region in merged_df['region'].unique():
    # Filter DataFrame for the current region
    region_df = merged_df[merged_df['region'] == region].copy()
    
    # Create a GeoDataFrame for the current region
    region_gdf = gpd.GeoDataFrame(region_df, geometry='geometry', crs=postcode_sectors_gdf.crs)
    
    # Store the GeoDataFrame in the dictionary under the region key
    region_geojson_dict[region] = region_gdf

# Now, region_geojson_dict contains GeoDataFrames for each region

for region, region_gdf in region_geojson_dict.items():
    region_geojson_path = f'geo_json/regions/{region}_postcode_sectors.geojson'
    region_gdf.to_file(region_geojson_path, driver='GeoJSON')

# Data analysis

Average Price by year


In [ ]:
# Group by region, postcode_sector, and year, then calculate the rounded average price and count the number of sales
grouped_df = df.groupby(['region', 'postcode_sector', 'year']).agg({'price': ['mean', 'count']}).reset_index()
grouped_df.columns = ['region', 'postcode_sector', 'year','avg_price', 'volume']

# Round the average price to the nearest thousand and convert to integers
grouped_df['avg_price'] = grouped_df['avg_price'].round(-3).astype(int)

# Loop through unique years and save CSV for each year
for year in grouped_df['year'].unique():
    # Filter DataFrame for the current year
    year_df = grouped_df[grouped_df['year'] == year]
    
    # Create the CSV file path for the current year
    csv_file_path = (f'processed_data/average_price_by_year/region_data_{year}.csv')
    
    # Save the filtered DataFrame to CSV
    year_df.to_csv(csv_file_path, index=False)

In [ ]:
grouped_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16382 entries, 0 to 16381
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   region           16382 non-null  object
 1   postcode_sector  16382 non-null  object
 2   year             16382 non-null  int32 
 3   avg_price        16382 non-null  int64 
 4   volume           16382 non-null  int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 576.1+ KB


Avg price for each region

In [ ]:
# Group by year, region, property_type, and calculate rounded average price
region_grouped_df = df.groupby(['year', 'region', 'property_type']).agg(avg_price=('price', 'mean')).round({'avg_price': -3}).astype({'avg_price': int}).reset_index()

# Calculate sales volume for each group
region_grouped_df['volume'] = df.groupby(['year', 'region', 'property_type']).size().reset_index(name='volume')['volume']

# Save the grouped data to a single CSV file
region_grouped_df.to_csv('processed_data/region_avg_price/region_avg_prices.csv', index=False)

Percentage delta of average price (year on year)

In [ ]:
# Group by region and postcode_sector, then calculate the percentage delta
delta_df = grouped_df.copy()  # Create a new DataFrame to store the results
delta_df['delta'] = grouped_df.groupby(['region', 'postcode_sector'])['avg_price'].pct_change() * 100

# Set the percentage change for the first year to 0
delta_df.loc[delta_df['year'] == delta_df['year'].min(), 'delta'] = 0

# Drop any rows with a null delta
delta_df = delta_df.dropna(subset=['delta'])

# Round values to the nearest integer
delta_df['delta'] = delta_df['delta'].round().astype(int)

# Get list of years from df
unique_years = delta_df['year'].unique()

# Iterate over each year and create a CSV file
for year in unique_years:
    # Filter rows for the current year
    year_df = delta_df[delta_df['year'] == year]

    # Create a CSV file for the current year
    csv_file_path = f'processed_data/avg_price_delta/avg_price_delta_{year}.csv'
    year_df.to_csv(csv_file_path, index=False)

Volume by month each year

In [ ]:
# Group by region, postcode_sector, year, and month, then calculate the count of sales
volume_df = df.groupby(['region', 'postcode_sector', 'year', 'month']).agg({'price': 'count'}).reset_index()

# Rename the columns for clarity
volume_df.columns = ['region', 'postcode_sector', 'year', 'month', 'volume']

# Loop through unique years
for year in volume_df['year'].unique():
    # Filter DataFrame for the current year
    year_df = volume_df[volume_df['year'] == year]
    
    # Group by region and month, then calculate the total volume
    total_volume_df = year_df.groupby(['region', 'month']).agg({'volume': 'sum'}).reset_index()
    
    # Create the CSV file path for the current year and save the DataFrame
    csv_file_path = f'processed_data/volume_by_year/region_total_volume_{year}.csv'
    total_volume_df.to_csv(csv_file_path, index=False)